In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal
from itertools import combinations
import matplotlib.patches as mpatches

import numpy as np
import os
import shutil
import cv2

from tqdm import trange, tqdm

# 
from scipy import signal

#    
from scipy.io import loadmat
import scipy

# 
import csv

from Analyze import plot_pairwise_interactions, ethogram_social_huddling
                    


Autosaving every 180 seconds


In [3]:
fnames_fixed = np.loadtxt('/home/cat/Downloads/Downloads/fnames_fixed.txt',dtype='str')
print (len(fnames_fixed))
models_fixed = np.loadtxt('/home/cat/Downloads/Downloads/models_fixed.txt',dtype='str')
print (len(models_fixed))

470
470


In [4]:

#def plot_pairwise_interactions2(locs_multi_hour):
def plot_pairwise_interactions2(locs_multi_hour):
    

    names= ['female','male','pup1','pup2']
    clrs = ['red','blue','cyan','green']

    x_ticks=['female','male','pup1','pup2']
    text_clr = 'red'

    distance_threshold = 250 # # of pixels away assume 1 pixel ~= 0.5mm -> 20cm
    time_window = 1*25 # no of seconds to consider
    smoothing_window = 3
    min_distance = 25 # number of frames window
    #

    # COMPUTE PAIRWISE INTERACTIONS
    pair_interaction_times2 = []
    triples_interaction_time2 = []
    quad_interaction_times2 = []
    multi_animal_durations2 = []
    interactions2 = []
    duration_matrix2 = []
    
    for locs in tqdm(locs_multi_hour):

        #
        locs = locs.transpose(1,0,2)
        #print ("locs: ", locs.shape)
        
        #
        traces_23hrs = locs
        
        animals=np.arange(locs.shape[0])
        interactions = np.zeros((animals.shape[0],animals.shape[0]),'int32') + np.nan
        durations_matrix = np.zeros((animals.shape[0], animals.shape[0]),'int32') + np.nan

        # loop over all pairwise combinations
        pair_interaction_times = []
        pairs1 = list(combinations(animals,2))
        for ctr_x, pair in enumerate(pairs1):
            traces = []

            # smooth out traces;
            for k in pair:
                traces1=traces_23hrs[k].copy()
                traces1[:,0]=np.convolve(traces_23hrs[k,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
                traces1[:,1]=np.convolve(traces_23hrs[k,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
                traces1 = traces1
                traces.append(traces1)

            # COMPUTE PAIRWISE DISTANCES AND NEARBY TIMES POINTS
            idx_array = []
            diffs = np.sqrt((traces[0][:,0]-traces[1][:,0])**2+
                            (traces[0][:,1]-traces[1][:,1])**2)
            idx = np.where(diffs<distance_threshold)[0]

            # COMPUTE TOTAL TIME TOGETHER
            #print ("Pairwise: ", pair, idx.shape)
            durations_matrix[pair[0],pair[1]]=idx.shape[0]/25

            # COMPUTE # OF INTERACTIONS;
            diffs_idx = idx[1:]-idx[:-1]
            idx2 = np.where(diffs_idx>5)[0]
            interactions[pair[0],pair[1]]=idx2.shape[0]

            # SAVE TIMES OF INTERACTION
            pair_interaction_times.append(idx)

        # SYMMETRIZE MATRICES
        for k in range(durations_matrix.shape[0]):
            for j in range(durations_matrix.shape[1]):
                if np.isnan(durations_matrix[k,j])==False:
                    durations_matrix[j,k]=durations_matrix[k,j]
                    interactions[j,k]=interactions[k,j]


        #############################
        # COMPUTE TRIPLE INTERACTIONS

        multi_animal_durations = [] #np.zeros((animals.shape[0]+4),'int32') + np.nan
        names_multi_animal =[]
        ctr=0

        pairs3 = list(combinations(animals,3))
        triples_interaction_times = []
        for ctr_x, pair in enumerate(pairs3):
            names_multi_animal.append(names[pair[0]]+"\n"+ names[pair[1]]+"\n"+names[pair[2]]+"\n")
            #names_multi_animal.append(names[pair[0]]+" - "+ names[pair[1]]+" - "+names[pair[2]])
            traces = []

            # COMPUTE LOCATIONS
            for k in pair:
                traces1=traces_23hrs[k].copy()
                traces1[:,0]=np.convolve(traces_23hrs[k,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
                traces1[:,1]=np.convolve(traces_23hrs[k,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
                traces1 = traces1
                traces.append(traces1)

            # COMPUTE PAIRWISE DISTANCES AND NEARBY TIMES POINTS
            idx_array = []
            pairs2 = list(combinations(np.arange(3),2))
            for pair_ in pairs2:
                #print ("pair_", pair_)
                diffs = np.sqrt((traces[pair_[0]][:,0]-traces[pair_[1]][:,0])**2+
                                (traces[pair_[0]][:,1]-traces[pair_[1]][:,1])**2)
                idx_temp = np.where(diffs<distance_threshold)[0]
                #print ("pair_: ", pair_, idx_temp.shape)
                idx_array.append(idx_temp)

            # COMPUTE OVERLAP
            idx3 = np.unique(np.hstack(set.intersection(*[set(x) for x in idx_array])))
            #print (pair, "IDX3 4: ", len(idx3))
            multi_animal_durations.append(len(idx3)/25)

            # SAVE TIMES OF INTERACTION
            triples_interaction_times.append(idx3)

        #print ("")

        #################################
        # COMPUTE 4 ANIMAL INTERACTIONS
        pairs = list(combinations(animals,4))
        quad_interaction_times = []
        for ctr_x, pair in enumerate(pairs):
            names_multi_animal.append(names[pair[0]]+"\n"+ names[pair[1]]+"\n"+names[pair[2]]+"\n"+names[pair[3]])
            #names_multi_animal.append(names[pair[0]]+" - "+ names[pair[1]]+" - "+names[pair[2]]+"  - "+names[pair[3]])
            #print (pair)
            traces = []
            for k in pair:
                traces1=traces_23hrs[k].copy()
                traces1[:,0]=np.convolve(traces_23hrs[k,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
                traces1[:,1]=np.convolve(traces_23hrs[k,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
                traces1 = traces1
                traces.append(traces1)

            # loop over all combinations and get distances
            pairs2 = list(combinations(animals,2))
            idx_array = []
            for pair2 in pairs2:
                diffs = np.sqrt((traces[pair2[0]][:,0]-traces[pair2[1]][:,0])**2+
                                 (traces[pair2[0]][:,1]-traces[pair2[1]][:,1])**2)
                idx_temp = np.where(diffs<distance_threshold)[0]
                #print ("pair2: ", pair2, idx_temp.shape)
                idx_array.append(idx_temp)

            # COMPUTE TOTAL TIME TOGETHER
            #print ("idx_array: ", idx_array)
            idx3 = set.intersection(*[set(x) for x in idx_array])
            #print ("4 animals 4: ", len(idx3))
            multi_animal_durations.append(len(idx3)/25)

            # SAVE TIMES OF INTERACTION
            quad_interaction_times.append(list(idx3)  )

            ctr+=1

        pair_interaction_times2.append(pair_interaction_times)
        triples_interaction_time2.append(triples_interaction_times)
        quad_interaction_times2.append(quad_interaction_times)
        multi_animal_durations2.append(multi_animal_durations)
        interactions2.append(interactions)
        duration_matrix2.append(durations_matrix)
    

    return (pair_interaction_times2,triples_interaction_time2, quad_interaction_times2, multi_animal_durations2, names_multi_animal, interactions2, duration_matrix2)


In [6]:
#
fnames_spine_fixed = np.loadtxt('/media/cat/4TBSSD/dan/cohort1/filenames_spine_fixed.txt',dtype='str')
print (len(fnames_spine_fixed))

# 
fnames_order = np.loadtxt('/media/cat/4TBSSD/dan/cohort1/filenames_reordered_clean.txt', dtype='str')

#
fname_out = '/media/cat/4TBSSD/dan/cohort1/all_locs.npy'


if os.path.exists(fname_out)==False:
    
    day_times = []
    #locs = np.zeros((90000*470,4,2), 'float32')
    locs = np.zeros((470,90000, 4,2), 'float32')
    print (locs.shape)
    shift =0
    hour=0
    for fname_order in tqdm(fnames_order):
        # 
        for ctr in range(len(fnames_spine_fixed)):
       
            if fname_order in fnames_spine_fixed[ctr]:
                # 
                #print (fnames_spine_fixed[ctr])
                temp = np.load(fnames_spine_fixed[ctr])#.transpose(1,0,2)
                # 
                #locs[shift:shift+temp.shape[0]] = temp
                if '2020_3_8_03_09_25_132594' in fname_order:
                    locs[hour,:temp.shape[0]//2] = temp[:temp.shape[0]//2]
                    hour+=1
                    locs[hour,:temp.shape[0]-temp.shape[0]//2] = temp[temp.shape[0]//2:]
                else:
                    locs[hour,:temp.shape[0]] = temp

                day_times.append(fname_order)
                    
                hour+=1
                
                break

print ("locs: ", locs.shape)
print ("day times: ", len(day_times))

  1%|          | 5/491 [00:00<00:10, 47.55it/s]

469
(470, 90000, 4, 2)


100%|██████████| 491/491 [00:12<00:00, 38.71it/s]

locs:  (470, 90000, 4, 2)
day times:  469


In [7]:
##################################################################
#################### EXTRACT PAIR INTERACTIONS ###################
##################################################################
# 
suptitle = "Behavior interactions 48hrs (March 15th-16th)"
suptitle = "20 days movement"
(pair_interaction_times2,
 triples_interaction_time2, 
 quad_interaction_times2, 
 multi_animal_durations2,
 names_multi_animal, 
 interactions2, 
 duration_matrix2) = plot_pairwise_interactions2(locs)


  0%|          | 0/470 [00:00<?, ?it/s]/media/cat/4TBSSD/anaconda3/envs/gerbil/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in less
/media/cat/4TBSSD/anaconda3/envs/gerbil/lib/python3.7/site-packages/ipykernel_launcher.py:107: RuntimeWarning: invalid value encountered in less
/media/cat/4TBSSD/anaconda3/envs/gerbil/lib/python3.7/site-packages/ipykernel_launcher.py:112: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/media/cat/4TBSSD/anaconda3/envs/gerbil/lib/python3.7/site-packages/ipykernel_launcher.py:143: RuntimeWarning: invalid value encountered in less
100%|██████████| 470/470 [00:50<00:00,  9.33it/s]


In [8]:
print (len(interactions2), len(day_times))
print (np.array(interactions2).shape)

470 469
(470, 4, 4)


In [12]:
###############################################################
#################### PLOT PAIR INTERACTIONS ###################
###############################################################
temp = np.array(interactions2)
print (temp.shape)
#print (interactions2.shape)
temp = temp.reshape(temp.shape[0],-1)
x_ticks=['female','male','pup1','pup2']

ctr=0
for k in range(16):
    ax=plt.subplot(4,4,ctr+1)
    #if k%4==0
    #    continue
        
    if k!=12:
        plt.xticks([])
        plt.yticks([])
    else:
        plt.xlabel("Time of day")
        plt.ylabel('Day of study')
        
    temp1 = temp[:,k]
    
    # 
    temp2 = []
    for p in range(0,temp1.shape[0],24):
        chunk = temp1[p:p+24]
        if chunk.shape[0]==24:
            temp2.append(chunk)
    temp2 = np.array(temp2)
    #print (temp2.shape)
    
    plt.title(x_ticks[k//4]+ " " + x_ticks[k%4])
    temp2 = np.roll(temp2,-5,axis=1)
    plt.imshow(temp2,vmin=0,vmax=1500,
               interpolation='none')
    ctr+=1
    

#plt.imshow(temp, aspect='auto', interpolation='none')
plt.suptitle("# of interactions per hour (0=min, 1500=max)")
plt.show()

(470, 4, 4)


In [ ]:

##############################################
############ PLOT PAIRWISE DURATIONS ########
#################################################
ax2=plt.subplot(1,2,2)
im = plt.imshow(durations_matrix, cmap='viridis')

x_ticks=['female','male','pup1','pup2']
plt.xticks(np.arange(locs.shape[0]), x_ticks,rotation=15)
plt.yticks(np.arange(locs.shape[0]), x_ticks,rotation=75)
plt.tick_params(labelsize=20)

cbar = plt.colorbar()
cbar.set_label("time together (sec)", fontsize=20)


# # Loop over data dimensions and create text annotations.
# if False:
#     for i in range(durations_matrix.shape[0]):
#         for j in range(durations_matrix.shape[1]):
#             if np.isnan(durations_matrix[i, j])==True:
#                 continue
#             text = ax2.text(j, i, durations_matrix[i, j],
#                            ha="center", va="center", color=text_clr,
#                            fontsize=13)

ax2.set_title('time together (sec)',fontsize=20)

plt.suptitle(suptitle, fontsize=15)

plt.show()

In [6]:
#######################################
####### TIME SPENT TOGETHER PLOTS #####
#######################################
ethogram_social_huddling(pair_interaction_times,
                         triples_interaction_time,
                         quad_interaction_times)


KeyboardInterrupt: 

In [8]:
############################
####### SMOOTH TRACES ######
############################
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

from tqdm import trange

box_width = 25
print (locs.shape)
locs_smooth = locs.copy()
for k in range(locs.shape[0]):
    for p in trange(locs.shape[2]):
        locs_smooth[k,:,p] = smooth(locs_smooth[k,:,p],box_width)


  0%|          | 0/2 [00:00<?, ?it/s]

(4, 4229429, 2)


100%|██████████| 2/2 [00:00<00:00,  5.49it/s]


In [9]:
##############################################################
####### FIND MOVEMENT INITIATIONS AND WHO FOLLOWS WHO ########
##############################################################
        
# compute velocities for each animal
vel = []
for k in range(locs_smooth.shape[0]):
    temp = locs_smooth[k]
    temp_vel = np.sqrt((temp[1:,0]-temp[:-1,0])**2+
                       (temp[1:,1]-temp[:-1,1])**2)
    vel.append(temp_vel)
    
vel = np.array(vel)


clrs = ['red','blue','cyan','green','magenta', 'black']
animal_names = ['female','male','pup1','pup2']

# detect movement from any period of rest
frame_rate = 25
min_rest_period = 5 * frame_rate

# walk vs. running thresholds
rest_velocity = 3  #very slowly moving / stationary threshold
                   # measured in pixels/frame movement; 
min_rest_duration = int(.25 * frame_rate)  # movement < n frames between rest will be discarded (i.e. counted as rest)
run_velocity = 30

# detect movement initiations from rest
mov_initiations = np.zeros((4, locs.shape[1]),'float32')
rest_array = []
for k in range(4):
    temp = vel[k]
    idx = np.where(temp<=rest_velocity)[0]
    print (k, idx.shape)
    
    # compute time differences for rest periods; search for periods > some duration to detect  rather than
    # artifactual trace jitter
    diff = idx[1:]-idx[:-1]
    
    rest_start = []
    rest_end = []
    rest_start.append(idx[0])
    for p in range(1,idx.shape[0],1):
        # check to see if moving to new rest state
        if (idx[p]-idx[p-1])>min_rest_duration:
            rest_end.append(idx[p-1])
            rest_start.append(idx[p])
    rest_end.append(idx[p])
    
    #
    rests = np.zeros((len(rest_start),2),'int32')
    rests[:,0] = rest_start
    rests[:,1] = rest_end
    rests = rests
    print (k, rests.shape)
    

    rest_array.append(rests)
    # connect all rests periods that are less than threshold apart:
    mov_initiations[k,rests[:,1]] += 1
    #for p in range(rests.shape[0]):
    
    # plot movement intiations as the end of rest periods:
    #plt.scatter(rests[:,1],rests[:,1]*0+k, c=clrs[k])
    
print (mov_initiations.shape)
#plt.show()

# PROCESS MOVEMENTS
fig = plt.figure()
frame_rate = 25
n_sec_window = 1

# make matrix to hold 10 second window of time. 
matrix = np.zeros((4,4),'float32')  # no. of animals, x no. of 
# loop over all animals
for k in range(len(rest_array)):
    # loop over each rest period
    for p in range(rest_array[k].shape[0]):
        # loop over all other animals        
        for a in range(4):
            diffs = rest_array[k][p,1]-rest_array[a][:,1]
            idx = np.where(diffs>=0)[0]
            #
            if idx.shape[0]>0:
                diffs=diffs[idx]
                # check 
                if np.min(diffs)<=(frame_rate*n_sec_window):
                    matrix[k,a]+=1


/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/anaconda3/envs/sleap/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in less_equal


0 (3331470,)
0 (12565, 2)
1 (3230966,)
1 (12399, 2)
2 (1591152,)
2 (19123, 2)
3 (1377801,)
3 (16620, 2)
(4, 4229429)


In [11]:
################################################################# 
#################################################################
#################################################################
#
for j in range(4):
    matrix[j,j]=np.nan
plt.imshow(matrix)
plt.colorbar()

x_ticks=['female','male','pup1','pup2']
plt.xticks(np.arange(4),x_ticks)
plt.yticks(np.arange(4),x_ticks)
plt.ylabel("Leader")
plt.xlabel("Follower")


fig = plt.figure()
for k in range(4):
    print (matrix[k]/rest_array[k].shape[0])
    plt.scatter(np.arange(4), 
                matrix[k]/rest_array[k].shape[0], 
                s=100,
                c=clrs[k], label=animal_names[k])

#x_ticks=['female','male','pup1','pup2']
plt.xticks(np.arange(4),x_ticks)

plt.legend(fontsize=20)
plt.ylim(bottom=0)
plt.title("Probability of following leader within "+ 
          str(n_sec_window) + 'sec' , fontsize=20)
plt.xlabel("Leader",fontsize=20)
plt.tick_params(labelsize=20)
plt.ylabel("Probability (%)", fontsize=20)
plt.show()


[       nan 0.11372861 0.17341822 0.14293674]
[0.11871925        nan 0.1696911  0.15081862]
[0.1023375  0.10887413        nan 0.15212049]
[0.10036101 0.10475331 0.18194945        nan]


In [ ]:
################################################################
####### FIND PROBABILITIES OF CHASING WITHIN 10 SEC OF MOVING ##
################################################################
# make matrix to hold 10 second window of time. 
matrix = np.zeros((4,11,4),'float32')  # no. of animals, x no. of animals, x 10 sec
# loop over all animals
for k in range(len(rest_array)):
    # loop over each rest period
    for p in range(rest_array[k].shape[0]):
        # loop over all other animals        
        for a in range(4):
            diffs = rest_array[k][p,1]-rest_array[a][:,1]
            idx = np.where(diffs>=0)[0]
            #
            if idx.shape[0]>0:
                diffs=diffs[idx]
                
                # check 
                #if np.min(diffs)<=(frame_rate*n_sec_window):
                # add all the values
                for q in range(0,11,1):
                    idx3 = np.where(np.logical_and(diffs>=(q*frame_rate),
                                                  diffs<((q+1)*frame_rate)))[0]
                    matrix[k,q,a]+=idx3.shape[0]
            
#
fig=plt.figure()
ax=plt.subplot()
fontsize=30
#for animal in range(4):
t=np.arange(11)+0.5
for animal in range(1):
    #ax=plt.subplot(4,1,animal+1)
    for k in range(4):
        if k==animal:
            continue
        plt.plot(t,matrix[animal,:,k]/rest_array[k].shape[0],
                 linewidth=5,
                label=animal_names[k] + " follows "+animal_names[animal])
    plt.legend(fontsize=fontsize)
    plt.xticks([])
    plt.xlim(0,10)
plt.xticks(np.arange(11))
plt.tick_params(labelsize=fontsize)
plt.ylim(bottom=0)
ax.axvspan(0,1, ymin=0, ymax=1, alpha=0.1, color='black')

plt.plot([0,10],[0.1, 0.1],'r--',linewidth=5, c='black')

plt.xlabel("Time (sec)", fontsize=fontsize)
plt.title("Predicting chase behaviour", fontsize=fontsize)
plt.show()